In [ ]:
# WSL
# cd /mnt/c/Users/rs659/Desktop/mmsearch

In [ ]:
# conda create -n mmsearchenv python=3.9
# conda activate mmsearchenv
# conda install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
# pip install -r requirements.txt
# docker-compose up -d

# https://anaconda.org/conda-forge/docker-compose


# docker rm -f $(docker ps -aq)


# conda install -c conda-forge poppler

In [ ]:
# https://developer.nvidia.com/cuda-12-2-0-download-archive?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local

In [ ]:
# https://github.com/weaviate-tutorials/DEMO-multimodal-search

In [ ]:
from pathlib import Path
import json

for f in Path("data/podcasts").glob("*.json"):  # assuming podcast data is in JSON files
    # podcast_data = json.loads(f.read_text())
    with open(f, 'r', encoding='utf-8') as file:
        podcast_data = json.load(file)

In [ ]:
len(podcast_data)

In [ ]:
# data_props = {
#     "title": podcast_data["title"],
#     "transcript": podcast_data["transcript"],
#     "description": podcast_data["description"],
#     "titleimage": base64.b64encode(Path(podcast_data["titleimage"]).read_bytes()).decode(),
# }

In [ ]:
from weaviate.util import generate_uuid5
from pathlib import Path
import json

data_objs = list()
podcast_dir = Path("data/podcasts")
for f in podcast_dir.glob("*.json"):  # assuming podcast data is in JSON files
    # podcast_data = json.loads(f.read_text())
    with open(f, 'r', encoding='utf-8') as file:
        podcast_data = json.load(file)
    for podcast in podcast_data:
        data_props = {
            "title": podcast["title"],
            "transcript": podcast["transcript"],
            # "description": podcast["description"],
            # "titleimage": base64.b64encode(Path(podcast["titleimage"]).read_bytes()).decode(),
        }
        data_obj = wvc.data.DataObject(
            properties=data_props, uuid=generate_uuid5(f.name)
        )
        data_objs.append(data_obj)

In [ ]:
import weaviate
import weaviate.classes as wvc
from weaviate.util import generate_uuid5
from weaviate import WeaviateClient
from weaviate.collections.classes.batch import BatchObjectReturn
import base64
from pathlib import Path

import json

def connect() -> WeaviateClient:
    return weaviate.connect_to_local()

client = connect()

In [ ]:
# mm_coll = client.collections.get('WineReviews') #some_objects = client.data_object.get()
# print(json.dumps(mm_coll))

# response = mm_coll.query.near_text(
#         query="",
#         limit=2
#     )


In [ ]:
mm_coll = client.collections.get('WineReviews')

response = mm_coll.aggregate.over_all(total_count=True)
print(f"Object count: {response.total_count}")

for q in ["lions"]:
    response = mm_coll.query.near_text(q, limit=3)
    print(f"\n{'*'*10} \t Query: \t {'*'*10}\n")
    print(f"\t\t {q} \n")
    print(f"{'*'*10} \t RESULTS: \t {'*'*10}\n")
    for r in response.objects:
        # print(r)
        print(r.properties["title"])

In [ ]:
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf(filename="data/pdf/paper01.pdf")

In [ ]:
# import textwrap

# narrative_texts = [elem for elem in elements if elem.category == "NarrativeText"]

# for index, elem in enumerate(narrative_texts[:5]):
#     print(f"Narrative text {index + 1}:")
#     print("\n".join(textwrap.wrap(elem.text, width=70)))
#     print("\n" + "-" * 70 + "\n")

In [ ]:
def define_collection_pdfs(client: WeaviateClient, collection_name: str = 'pdfs') -> bool:

    # Creating a new collection with the defined schema
    client.collections.create(
        name=collection_name,
        properties=[
            wvc.Property(
                name="source",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="abstract",
                data_type=wvc.DataType.TEXT,
            )
        ],
        vectorizer_config=wvc.config.Configure.Vectorizer.multi2vec_bind(
            text_fields=[wvc.config.Multi2VecField(name='title', weight=0.95)],
            vectorize_collection_name=False )
    )

client.collections.delete('pdf')
define_collection_pdfs(client,'pdf')


In [ ]:
import logging

logging.basicConfig(level=logging.INFO)


class AbstractExtractor:
    def __init__(self):
        self.current_section = None  # Keep track of the current section being processed
        self.have_extracted_abstract = (
            False  # Keep track of whether the abstract has been extracted
        )
        self.in_abstract_section = (
            False  # Keep track of whether we're inside the Abstract section
        )
        self.texts = []  # Keep track of the extracted abstract text

    def process(self, element):
        if element.category == "Title":
            self.set_section(element.text)

            if self.current_section == "Abstract":
                self.in_abstract_section = True
                return True

            if self.in_abstract_section:
                return False

        if self.in_abstract_section and element.category == "NarrativeText":
            self.consume_abstract_text(element.text)
            return True

        return True

    def set_section(self, text):
        self.current_section = text
        logging.info(f"Current section: {self.current_section}")

    def consume_abstract_text(self, text):
        logging.info(f"Abstract part extracted: {text}")
        self.texts.append(text)

    def consume_elements(self, elements):
        for element in elements:
            should_continue = self.process(element)

            if not should_continue:
                self.have_extracted_abstract = True
                break

        if not self.have_extracted_abstract:
            logging.warning("No abstract found in the given list of objects.")

    def abstract(self):
        return "\n".join(self.texts)

In [ ]:
data_folder = "data/pdf/"

data_objects = []

for path in Path(data_folder).iterdir():
    if path.suffix != ".pdf":
        continue

    # print(f"Processing {path.name}...")

    elements = partition_pdf(filename=path)

    abstract_extractor = AbstractExtractor()
    abstract_extractor.consume_elements(elements)

    data_object = {"source": path.name, "abstract": abstract_extractor.abstract()}

    data_objects.append(data_object)

In [ ]:
# data_objects

In [18]:
# client.batch.configure(batch_size=100)  # Configure batch
# with client.batch as batch:
#     for data_object in data_objects:
#         batch.add_data_object(data_object, "Document")

pdf_collection = client.collections.get('pdf')

pdf_collection.data.insert_many(data_objects)

BatchObjectReturn(all_responses=[UUID('ecd88190-bd63-49c8-983a-c8962f79dd2f'), UUID('299eea7c-f816-46c8-8589-bfc82497e23f')], elapsed_seconds=0.002997875213623047, errors={}, uuids={0: UUID('ecd88190-bd63-49c8-983a-c8962f79dd2f'), 1: UUID('299eea7c-f816-46c8-8589-bfc82497e23f')}, has_errors=False)

In [24]:
response = pdf_collection.query.fetch_objects(
    include_vector=True,
    limit=1,
    offset=1
)

for o in response.objects:
    print(o.properties)

{'abstract': 'Purchasing a home is one of the largest investments most people make. House price prediction allows indi- viduals to be informed about their asset wealth. Trans- parent pricing on homes allows for a more efficient mar- ket and economy. We report the performance of ma- chine learning models trained with structured tabular representations and unstructured text descriptions. We collected a dataset of 200 descriptions of houses which include meta-information, as well as text descriptions. We test logistic regression and multi-layer perceptron (MLP) classifiers on dividing these houses into binary buckets based on fixed price thresholds. We present an exploration into strategies to represent unstructured text descriptions of houses as inputs for machine learning models. This includes a comparison of term frequency- inverse document frequency (TF-IDF), bag-of-words (BoW), and zero-shot inference with large language models. We find the best predictive performance with TF-IDF rep

In [25]:
response = pdf_collection.query.bm25(
     query="largest",
    limit=3
)

for o in response.objects:
    print(o.properties)

{'abstract': 'Purchasing a home is one of the largest investments most people make. House price prediction allows indi- viduals to be informed about their asset wealth. Trans- parent pricing on homes allows for a more efficient mar- ket and economy. We report the performance of ma- chine learning models trained with structured tabular representations and unstructured text descriptions. We collected a dataset of 200 descriptions of houses which include meta-information, as well as text descriptions. We test logistic regression and multi-layer perceptron (MLP) classifiers on dividing these houses into binary buckets based on fixed price thresholds. We present an exploration into strategies to represent unstructured text descriptions of houses as inputs for machine learning models. This includes a comparison of term frequency- inverse document frequency (TF-IDF), bag-of-words (BoW), and zero-shot inference with large language models. We find the best predictive performance with TF-IDF rep

In [36]:
response = pdf_collection.query.hybrid(
     query="some thing about ghar",
    limit=3
)

# for o in response.objects:
#     print(o.properties)


type(response.objects)

list

In [ ]:
result = (
    client.query
    .get("Question", ["question", "answer", "category"])
    # .with_near_text(nearText)
    .with_limit(2)
    .with_additional(['certainty'])
    .do()
)

pdf_collection.query.near_text("some paper about housing prices", limit=3)